# Requirements

Since this notebook uses a Julia package that is not in the standard library, an environment was created in which it is installed.  Using the `Pkg` module, that environment, `struct_arrrays`, can be activated, so that the `StructArrays` module can be loaded.

In [1]:
using Pkg
Pkg.activate("struct_arrays")
Pkg.instantiate()
using StructArrays
using BenchmarkTools

  Activating project at `~/Projects/Julia_good_bad_ugly/source-code/ecosystem/struct-arrays/struct_arrays`
[ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]


# Array of structures

Consider a structure that represents a particle.  A point has an $x$ and $y$ coordinate, and a mass.

In [2]:
struct Particle
    x::Float64
    y::Float64
    mass::Float64
    v_x::Float64
    v_y::Float64
    charge::Int32
end

We create many such particles and store them in a `Vector`.

In [3]:
nr_particles = 10_000_000
particles = Array{Particle}(undef, nr_particles)
for i in eachindex(particles)
    particles[i] = Particle(randn(), randn(), rand(), randn(), randn(), 1)
end

In [4]:
typeof(particles)

Vector{Particle} (alias for Array{Particle, 1})

We want to compute the center of mass and define a function to compute it.

In [5]:
function center_of_mass(particles::Array{Particle})
    x_coord = sum(map(p -> p.x, particles))
    y_coord = sum(map(p -> p.y, particles))
    mass = sum(map(p -> p.mass, particles))
    return x_coord/mass, y_coord/mass
end

center_of_mass (generic function with 1 method)

In [6]:
@benchmark center_of_mass(particles)

BenchmarkTools.Trial: 38 samples with 1 evaluation.
 Range (min … max):  111.231 ms … 285.693 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     122.529 ms               ┊ GC (median):    5.69%
 Time  (mean ± σ):   134.168 ms ±  35.907 ms  ┊ GC (mean ± σ):  3.95% ± 3.29%

     █                                                           
  ▃▄▄█▇▇▃▁▅▃▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▃ ▁
  111 ms           Histogram: frequency by time          286 ms <

 Memory estimate: 228.88 MiB, allocs estimate: 7.

The memory access pattern in the previous implementation is far from optimal, for the current data stucture it can be improved by iterating over the particles and computing the $x$ and $y$ coordinates and the total mass one particle at th4e time.

In [7]:
function center_of_mass_iter(particles::Array{Particle})
    x_coord, y_coord, mass = 0.0, 0.0, 0.0
    for particle in particles
        x_coord += particle.x
        y_coord += particle.y
        mass += particle.mass
    end
    return x_coord/mass, y_coord/mass
end

center_of_mass_iter (generic function with 1 method)

In [8]:
@benchmark center_of_mass_iter(particles)

BenchmarkTools.Trial: 231 samples with 1 evaluation.
 Range (min … max):  20.889 ms … 35.055 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     21.347 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.626 ms ±  1.317 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▇█▇▆▅▃▁                                                     
  █████████▆█▆▁▇▁▄▆▄▁▁▁▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▆
  20.9 ms      Histogram: log(frequency) by time      28.4 ms <

 Memory estimate: 32 bytes, allocs estimate: 1.

# Structure of arrays

However, note that we are accessing the `x`, `y` and `mass` field of each particle one after the other.  Storing these fields in arrays would improve the memory access pattern.

In [9]:
particles_sa = StructArray(particles);

In [10]:
typeof(particles_sa)

StructVector{Particle, NamedTuple{(:x, :y, :mass, :v_x, :v_y, :charge), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Int32}}}, Int64} (alias for StructArray{Particle, 1, NamedTuple{(:x, :y, :mass, :v_x, :v_y, :charge), Tuple{Array{Float64, 1}, Array{Float64, 1}, Array{Float64, 1}, Array{Float64, 1}, Array{Float64, 1}, Array{Int32, 1}}}, Int64})

In [11]:
function center_of_mass(particles::StructArray{Particle})
    x_coord = sum(particles.x)
    y_coord = sum(particles.y)
    mass = sum(particles.mass)
    return x_coord/mass, y_coord/mass
end

center_of_mass (generic function with 2 methods)

In [12]:
@benchmark center_of_mass(particles_sa)

BenchmarkTools.Trial: 461 samples with 1 evaluation.
 Range (min … max):  10.492 ms …  13.908 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.799 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.826 ms ± 222.793 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▃▁▂▁▃▂█▂▆▄▃▂ ▃                                      
  ▂▁▃▃▂▃▄▄▅▇████████████▇█▅▅▅▅▆▃▃▃▃▃▃▃▃▃▁▁▂▁▁▂▂▁▃▁▂▁▂▁▁▁▁▁▁▁▁▂ ▃
  10.5 ms         Histogram: frequency by time         11.5 ms <

 Memory estimate: 32 bytes, allocs estimate: 1.

This data structure is much more suited for the memory access pattern in our original implementation.  All $x$ coordinates are stored continguously in memory, and similar for $y$ and the masses.